This cell is for running DeepCFR training on two methods of sampling and two different environements.

In [ ]:
import pyspiel
import open_spiel
import pandas as pd
import numpy as np
import copy
from cfr_utils import WrapperEnv
from agent_configs.cfr_config import CFRConfig
from active_player import ActivePlayer
from cfr_agent import CFRAgent
import torch
from cfr_network import CFRNetwork
fhp = pyspiel.load_game("universal_poker", {"numPlayers":2, "numSuits": 4, "numRanks":13, "numHoleCards": 2, "numBoardCards": "0 3", "bettingAbstraction": "fcpa", "numRounds":2, "blind": "50 100"})
leduc = pyspiel.load_game("universal_poker", {"numPlayers":2, "numSuits": 2, "numRanks":3, "numHoleCards": 1, "numBoardCards": "0 1", "bettingAbstraction": "fcpa", "numRounds":2, "blind": "50 100"})
leducgame = WrapperEnv(leduc)
fhpgame = WrapperEnv(fhp)
games = [leducgame, fhpgame]
hidden_dim = 128
input_dim = 108
output_dim = 4
num_players = 2
replay_buffer_size = 4000000
minibatch_size = 10000
steps_per_epoch = 3000
traversals = 3000
training_steps = 20000
lr = 0.001
optimizer = None
sampling = ["Full", "MC"]

In [ ]:
for i in sampling:
    for o in games:
        if o == fhpgame:
            insize = 108
            outsize = 4
            modelname = "CFR_0.001LR_TEST_" + i + "_FHP"
        elif o == leducgame:
            insize = 16
            outsize = 4
            modelname = "CFR_0.001LR_TEST_" + i + "_Leduc"
        p_v_networks = {'input_shape':insize, 'output_shape':outsize, 'hidden_size':hidden_dim, 'learning_rate':lr, 'optimizer':optimizer}
        active_player_obj = ActivePlayer(num_players)
        config = CFRConfig(
            config_dict={'network': {'policy': p_v_networks, 'value': p_v_networks, 'num_players':num_players},
                        'replay_buffer_size':replay_buffer_size,
                        'minibatch_size':minibatch_size,
                        'steps_per_epoch':steps_per_epoch,
                        'traversals': traversals,
                        'training_steps': training_steps,
                        'active_player_obj': active_player_obj,
                        },
            game_config={'num_players':num_players,
                        'observation_space':insize,
                        'action_space':outsize,},
        )

        model = CFRAgent(env=o,config=config, name=modelname, max_nodes=10000000)
        model.train(sampling=i)